# [Advent of Code 2022 Day 10](https://adventofcode.com/2022/day/10)

I got tripped by this one and spent 40 minutes dealing with a bug that wouldn't have existed had I read the question correctly...

## Initial setup

In [1]:
import doctest

import ipytest
import pytest
import sys
sys.path.append("..")
from ansi import *
from comp import *
ipytest.autoconfig()
PART_ONE_SENTINEL = 0x3f3f3f3f + 1
PART_TWO_SENTINEL = 0x3f3f3f3f + 2
run_doctest_for = lambda func: doctest.run_docstring_examples(func, globals())

## Part 1 Test Cases

In [2]:
PART_ONE_CASES: dict[str, dict[str, str | int]] = {
    "example": {
        "example1": PART_ONE_SENTINEL,
        "example2": 13140,
    },
    "input": {
        "input1": 12740,
    },
}
PART_ONE_OUTPUTS: dict[str, dict[str, str | int]] = {
    key: {} for key in PART_ONE_CASES.keys()
}

## Part 2 Test Cases

In [3]:
PART_TWO_CASES: dict[str, dict[str, str | int]] = {
    "example": {
        "example1": PART_TWO_SENTINEL,
    },
    "input": {
        "input1": PART_TWO_SENTINEL,
    },
}
PART_TWO_OUTPUTS: dict[str, dict[str, str | int]] = {
    key: {} for key in PART_TWO_CASES.keys()
}

## Input Parsing

In [4]:
def parse_input(ctx: Context) -> Context:
    gen = yield_line(ctx.filename)

    ctx = Context()
    ctx.input = []

    input_lines = ctx.input

    for idx, line in enumerate(gen):
        if line == "noop":
            input_lines.append(("noop", None))
            continue
        addx, value = parse(r"(addx) (-?\d+)", line)
        input_lines.append((addx, int(value)))

    return ctx

In [5]:
%%ipytest -xrPvvvvv
@pytest.mark.parametrize("test_file_name", PART_ONE_CASES["example"].keys() | PART_TWO_CASES["example"].keys())
def test_parsing_examples(test_file_name):
    ctx = Context()
    ctx.filename = test_file_name
    print(parse_input(ctx).input)

..                                                                                           [100%]
============================================= PASSES ==============================================
_________________________________ test_parsing_examples[example1] _________________________________
-------------------------------------- Captured stdout call ---------------------------------------
[('noop', None), ('addx', 3), ('addx', -5)]
_________________________________ test_parsing_examples[example2] _________________________________
-------------------------------------- Captured stdout call ---------------------------------------
[('addx', 15), ('addx', -11), ('addx', 6), ('addx', -3), ('addx', 5), ('addx', -1), ('addx', -8), ('addx', 13), ('addx', 4), ('noop', None), ('addx', -1), ('addx', 5), ('addx', -1), ('addx', 5), ('addx', -1), ('addx', 5), ('addx', -1), ('addx', 5), ('addx', -1), ('addx', -35), ('addx', 1), ('addx', 24), ('addx', -19), ('addx', 1), ('addx', 16), ('addx', 

In [6]:
%%ipytest -xrPvvvvv
@pytest.mark.parametrize("test_file_name", PART_ONE_CASES["input"].keys() | PART_TWO_CASES["input"].keys())
def test_parsing_inputs(test_file_name):
    ctx = Context()
    ctx.filename = test_file_name
    print(parse_input(ctx).input)

.                                                                                            [100%]
============================================= PASSES ==============================================
___________________________________ test_parsing_inputs[input1] ___________________________________
-------------------------------------- Captured stdout call ---------------------------------------
[('noop', None), ('noop', None), ('noop', None), ('addx', 4), ('addx', 1), ('addx', 5), ('addx', 1), ('addx', 5), ('noop', None), ('addx', -1), ('addx', -6), ('addx', 11), ('noop', None), ('noop', None), ('noop', None), ('noop', None), ('addx', 6), ('addx', 5), ('noop', None), ('noop', None), ('noop', None), ('addx', -30), ('addx', 34), ('addx', 2), ('addx', -39), ('noop', None), ('addx', 5), ('addx', 2), ('addx', 19), ('addx', -18), ('addx', 2), ('addx', 5), ('addx', 2), ('addx', 3), ('noop', None), ('addx', 2), ('addx', 3), ('noop', None), ('addx', 2), ('addx', 3), ('noop', None), ('addx', 2

## Sandbox

In [7]:
%%ipytest -xrPvvvvv

def log(message: str, newline: bool = True) -> None:
    print(f"{message}", file=sys.stderr, end="\n" if newline else "")

def get_signals(instructions: list[tuple[str, int | None]]) -> list[int]:
    """
    >>> get_signals([("noop", None), ("addx", 3), ("addx", -5)])
    [0, 1, 1, 1, 4, 4]
    """
    signals = [0]
    curr_cycle = 1
    pc = 0
    work = None
    x = 1

    while True:
        log("")
        log(f"Cycle {curr_cycle} -> ", False)
        curr_cycle += 1
        signals.append(x)
        if pc >= len(instructions):
            log(f"No more instructions left while x was {x} -> ", False)
            if work is not None:
                log(f"Catching up on last work {work} and adding it to {x} before ending run", False)
                x += work
            else:
                log(f"No work left either -> Ending entire run", False)
            break
        word, amt = instructions[pc]
        if word == "noop":
            if work is not None:
                log(f"Got noop while x was {x}. Catching up on latent work -> added {work} to {x} -> {x + work}", False)
                x += work
                work = None
            else:
                log(f"Got noop while x was {x} but do not have work to catch up on. Advancing pc", False)
                pc += 1
        elif word == "addx":
            if work is not None:
                log(f"Received addx {amt} while x was {x} but currently working on adding {work} to {x}... delaying -> ", False)
                x += work
                log(f"x is now {x}", False)
                work = None
            else:
                work = amt
                log(f"Received addx {amt} while x was {x}. No work is currently being done, so starting work on adding {work} to {x} -> should be done next cycle", False)
                pc += 1
        else:
            raise Exception("invalid branch")

    return signals

def test_get_signals():
    assert get_signals([("noop", None), ("addx", 3), ("addx", -5)]) == [0, 1, 1, 1, 4, 4]

def test_example2_checkpoints() -> None:
    parse_ctx = Context()
    parse_ctx.filename = "example2"
    ctx = parse_input(parse_ctx)
    signals = get_signals(ctx.input)
    assertions = (
        (20, 21),
        (60, 19),
        (100, 18),
        (140, 21),
        (180, 16),
        (220, 18),
    )
    for cycle_number, expected in assertions:
        assert signals[cycle_number] == expected, f"expected the value at {cycle_number=} to equal {expected} but got {signals[cycle_number]}"

..                                                                                           [100%]
============================================= PASSES ==============================================
________________________________________ test_get_signals _________________________________________
-------------------------------------- Captured stderr call ---------------------------------------

Cycle 1 -> Got noop while x was 1 but do not have work to catch up on. Advancing pc
Cycle 2 -> Received addx 3 while x was 1. No work is currently being done, so starting work on adding 3 to 1 -> should be done next cycle
Cycle 3 -> Received addx -5 while x was 1 but currently working on adding 3 to 1... delaying -> x is now 4
Cycle 4 -> Received addx -5 while x was 4. No work is currently being done, so starting work on adding -5 to 4 -> should be done next cycle
Cycle 5 -> No more instructions left while x was 4 -> Catching up on last work -5 and adding it to 4 before ending run
___________

In [8]:
def get_pixels(pos: int) -> list[int]:
    """
    >>> get_pixels(0)
    [0, 1]
    >>> get_pixels(39)
    [38, 39]
    >>> get_pixels(20)
    [19, 20, 21]
    >>> get_pixels(-100)
    []
    >>> get_pixels(200)
    []
    """
    if not (0 <= pos <= 39):
        return []
    if pos == 0:
        return [0, 1]
    if pos == 39:
        return [38, 39]
    return [pos - 1, pos, pos + 1]

run_doctest_for(get_pixels)

In [9]:
lmao_ctx = Context()
lmao_ctx.filename = "example2"
lmao = get_signals(parse_input(lmao_ctx).input)

lmao_ctx2 = Context()
lmao_ctx2.filename = "input1"
lmao2 = get_signals(parse_input(lmao_ctx2).input)

def draw(signals: list[int]) -> None:
    """
    >>> draw(lmao)
    ##..##..##..##..##..##..##..##..##..##..
    ###...###...###...###...###...###...###.
    ####....####....####....####....####....
    #####.....#####.....#####.....#####.....
    ######......######......######......####
    #######.......#######.......#######.....
    >>> draw(lmao2)
    ###..###..###...##..###...##...##..####.
    ...#.#..#.#..#.#..#.#..#.#..#.#..#.#....
    ...#.###..#..#.#..#.#..#.#..#.#....###..
    ###..#..#.###..####.###..####.#.##.#....
    ..#..#..#.#....#..#.#.#..#..#.#..#.#....
    ...#.###..#....#..#.#..#.#..#..###.#....
    """

    image = []
    for _ in range(6):
        image.append(["x"] * 40)

    for cycle_number in range(1, len(signals) - 1):
        pixel_idx_to_draw = cycle_number - 1
        x, y = divmod(pixel_idx_to_draw, 40)
        pixels = get_pixels(signals[cycle_number])
        if y in pixels:
            image[x][y] = "#"
        else:
            image[x][y] = "."

    for row in image:
        print("".join(row))

run_doctest_for(draw)


Cycle 1 -> Received addx 15 while x was 1. No work is currently being done, so starting work on adding 15 to 1 -> should be done next cycle
Cycle 2 -> Received addx -11 while x was 1 but currently working on adding 15 to 1... delaying -> x is now 16
Cycle 3 -> Received addx -11 while x was 16. No work is currently being done, so starting work on adding -11 to 16 -> should be done next cycle
Cycle 4 -> Received addx 6 while x was 16 but currently working on adding -11 to 16... delaying -> x is now 5
Cycle 5 -> Received addx 6 while x was 5. No work is currently being done, so starting work on adding 6 to 5 -> should be done next cycle
Cycle 6 -> Received addx -3 while x was 5 but currently working on adding 6 to 5... delaying -> x is now 11
Cycle 7 -> Received addx -3 while x was 11. No work is currently being done, so starting work on adding -3 to 11 -> should be done next cycle
Cycle 8 -> Received addx 5 while x was 11 but currently working on adding -3 to 11... delaying -> x is now 

In [10]:
def solve(part: int, filename: str) -> int:
    parse_ctx = Context()
    parse_ctx.filename = filename
    ctx = parse_input(parse_ctx)
    if part == 1:
        if filename == "example1":
            return PART_ONE_SENTINEL
        s = get_signals(ctx.input)
        ops = {20, 60, 100, 140, 180, 220}
        res = 0
        for op in ops:
            res += (op * s[op])
        return res
    if part == 2:
        # draw(ctx.input)
        return PART_TWO_SENTINEL
    else:
        raise Exception(f"Invalid part: {part}")

## Part 1

In [11]:
%%ipytest -xrPvvvvv
@pytest.mark.parametrize("test_file_name, test_expected_output", PART_ONE_CASES["example"].items())
def test_part_one_examples(test_file_name, test_expected_output):
    test_actual_output = solve(1, test_file_name)
    PART_ONE_OUTPUTS["example"][test_file_name] = test_actual_output
    failure_message = "Did you forget to calibrate the example test case?" if (
        test_expected_output == PART_ONE_SENTINEL
    ) else f"Failed example test case: expected {test_expected_output} but got {test_actual_output}"
    assert test_actual_output == test_expected_output, failure_message

@pytest.mark.parametrize("test_file_name, test_expected_output", PART_ONE_CASES["input"].items())
def test_part_one_inputs(test_file_name, test_expected_output):
    test_actual_output = solve(1, test_file_name)
    PART_ONE_OUTPUTS["input"][test_file_name] = test_actual_output
    failure_message = f"Candidate answer {test_actual_output} found" if (
        test_expected_output == PART_ONE_SENTINEL
    ) else f"Failed input test case: expected {test_expected_output} but got {test_actual_output}"
    assert test_actual_output == test_expected_output, failure_message

...                                                                                          [100%]
============================================= PASSES ==============================================
_____________________________ test_part_one_examples[example2-13140] ______________________________
-------------------------------------- Captured stderr call ---------------------------------------

Cycle 1 -> Received addx 15 while x was 1. No work is currently being done, so starting work on adding 15 to 1 -> should be done next cycle
Cycle 2 -> Received addx -11 while x was 1 but currently working on adding 15 to 1... delaying -> x is now 16
Cycle 3 -> Received addx -11 while x was 16. No work is currently being done, so starting work on adding -11 to 16 -> should be done next cycle
Cycle 4 -> Received addx 6 while x was 16 but currently working on adding -11 to 16... delaying -> x is now 5
Cycle 5 -> Received addx 6 while x was 5. No work is currently being done, so starting work on 

## Part 2

In [12]:
%%ipytest -xrPvvvvv
@pytest.mark.parametrize("test_file_name, test_expected_output", PART_TWO_CASES["example"].items())
def test_part_two_examples(test_file_name, test_expected_output):
    test_actual_output = solve(2, test_file_name)
    PART_TWO_OUTPUTS["example"][test_file_name] = test_actual_output
    failure_message = "Did you forget to calibrate the example test case?" if (
            test_expected_output == PART_TWO_SENTINEL
    ) else f"Failed example test case: expected {test_expected_output} but got {test_actual_output}"
    assert test_actual_output == test_expected_output, failure_message

@pytest.mark.parametrize("test_file_name, test_expected_output", PART_TWO_CASES["input"].items())
def test_part_two_inputs(test_file_name, test_expected_output):
    test_actual_output = solve(2, test_file_name)
    PART_TWO_OUTPUTS["input"][test_file_name] = test_actual_output
    failure_message = f"Candidate answer {test_actual_output} found" if (
            test_expected_output == PART_TWO_SENTINEL
    ) else f"Failed input test case: expected {test_expected_output} but got {test_actual_output}"
    assert test_actual_output == test_expected_output, failure_message

..                                                                                           [100%]
============================================= PASSES ==============================================
2 passed in 0.02s
